In [1]:
from oct2py import Oct2Py

import os
import pandas as pd
import csv

In [2]:
V_S = 7.0                   # service speed [kn]

In [3]:
n_runs = 3 # number of executions 

In [4]:
seed = 1

Fitness and Evaluation

In [7]:
# original dif evo
dif_evo = 0

# modified fitness dif evo
# dif_evo = 1

In [8]:
# run the Octave script to get the power brake for the params
def run_original(V_S, seed):
    P_B, t075dD,tmin075dD, tal07R,cavLim, Vtip,Vtipmax = [-1, -1, -1, -1, -1, -1, -1]
    output_filename = 'df_' + str(seed) + '.csv'
    with Oct2Py() as octave:
        octave.warning ("off", "Octave:data-file-in-path");
        octave.addpath('./allCodesOctave');
        octave.eval('pkg load statistics')
        if dif_evo == 0:
            D, Z, AEdAO, PdD, P_B = octave.F_DifEvo_LH2_return_constraints(V_S, output_filename, nout=5)
        elif dif_evo == 1:
            D, Z, AEdAO, PdD, P_B = octave.F_DifEvo_LH2_return_constraints_fitness(V_S, output_filename, nout=5)
    return [P_B, t075dD,tmin075dD, tal07R,cavLim, Vtip,Vtipmax]

In [9]:
run_original(V_S, seed)

k = 1
of = 1
of = 2
of = 3
of = 4
of = 5
k = 2
of = 6
of = 7
of = 8
of = 9
of = 10
k = 3
of = 11
of = 12
of = 13
of = 14
of = 15
k = 4
of = 16
of = 17
of = 18
of = 19
of = 20
k = 5
of = 21
of = 22
of = 23
of = 24
of = 25
best
X =

   0.6032   5.0000   0.5757   0.7893
   0.7768   3.0000   0.3414   0.7653
   0.7199   4.0000   0.3409   1.0954
   0.5105   3.0000   0.8280   1.0484
   0.7471   3.0000   0.5470   0.9245

VF =

   1.0368e+02   1.0972e+03   3.0476e-01   1.0247e+00
   8.3390e+01   7.4548e+02   3.7970e-01   1.0262e+00
   9.8476e+01   6.9440e+02   3.2784e-01   1.0053e+00
   1.3869e+02   1.2693e+03   2.3107e-01   1.0083e+00
   9.4979e+01   6.9957e+02   3.3649e-01   1.0161e+00

D = 0.7768
Z = 3
AEdAO = 0.3414
PdD = 0.7653
P_B = 83.390


[83.39020284279947, -1, -1, -1, -1, -1, -1]

## Make CSV

In [10]:
if dif_evo == 0:
    dir_to_save = 'original'
elif dif_evo == 1:
    dir_to_save = 'original_mod'
    
dir_to_save += str(seed)

In [29]:
def get_best_fit_number(x):
    for i in range(len(x)):
        x.iloc[i] = x.iloc[i][len('best fit at iteration '):]
    return x

col_names_original = ['D', 'Z', 'AEdAO', 'PdD', 'P_B', 'n', 'etaO', 'etaR', 't075dD','tmin075dD', 'tal07R','cavLim', 'Vtip','Vtipmax', 'fitness', 'iteration', 'population_i']
col_names = ['D', 'AEdAO', 'PdD', 'Z', 'P_B', 'n', 'fitness', 't075dD','tmin075dD', 'tal07R','cavLim', 'Vtip','Vtipmax', 'penalty', 'valid']

filename = 'df_' + str(seed) + '.csv'

df = pd.read_csv(filename, header=None, skiprows=1, names=col_names_original)

df = df[['D', 'AEdAO', 'PdD', 'Z', 'P_B', 'n', 'fitness', 't075dD','tmin075dD', 'tal07R','cavLim', 'Vtip','Vtipmax']]

# -- Calculate validity --

# insert the other collumns
df.insert(df.shape[1], 'penalty', None)

df.insert(df.shape[1], 'valid', True)

# create temporary
df['temp_cav'] = (df['tal07R'] - df['cavLim'])/df['cavLim']
df['temp_cav'] = df['temp_cav'].clip(lower=0)
df['temp_spd'] = (df['Vtip'] - df['Vtipmax'])/df['Vtipmax']
df['temp_spd'] = df['temp_spd'].clip(lower=0)
df['temp_str'] = (df['tmin075dD'] - df['t075dD'])/df['tmin075dD']
df['temp_str'] = df['temp_str'].clip(lower=0)
# calculate
df['penalty'] = df['temp_cav'] + df['temp_spd'] + df['temp_str']
df['valid'] = (df['penalty'] == 0)
# drop temp
df = df.drop(['temp_cav', 'temp_spd', 'temp_str'], axis=1)


# ajust the iteration counter with fitness
df['D'] = df['D'].astype(str)
df.loc[df['D'].str.contains('best'), 'fitness'] = df.loc[df['D'].str.contains('best'), 'Z']
df.loc[df['D'].str.contains('best'), 'Z'] = None

df.loc[df['D'].str.contains('best'), 'AEdAO'] = get_best_fit_number(df.loc[df['D'].str.contains('best'), 'D'])
df.loc[df['D'].str.contains('best'), 'D'] = 'fitness at iteration'

# -- add best value to the end --
best_fit_to_find = df.loc[len(df)-1]['fitness']
result_row = df[(df['P_B'] == best_fit_to_find) & (df['D'] != 'fitness at iteration')] # find row with best fitness?
df.loc[len(df)] = {'D': 'Best Solution'} # add text to the bottom
df.loc[len(df)] = result_row.iloc[0] # add row with values in the end

# # Check if the directory already exists
if not os.path.exists(dir_to_save):
    # Create the directory if it doesn't exist
    os.makedirs(dir_to_save)
    print(f"Directory '{dir_to_save}' created.")

filename_out = dir_to_save+'/'+str(seed)+'.csv'
df.to_csv(filename_out, index=False, header=True)

print('saved:', filename_out)
df.tail(10)

saved: original/1.csv


/tmp/ipykernel_39512/3483685775.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '2' '3' '4' '5']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['D'].str.contains('best'), 'AEdAO'] = get_best_fit_number(df.loc[df['D'].str.contains('best'), 'D'])


,D,AEdAO,PdD,Z,P_B,n,fitness,t075dD,tmin075dD,tal07R,cavLim,Vtip,Vtipmax,penalty,valid
27,0.500000,0.54701,0.924501,3.0,0.000000,1493.975810,0.000000,0.01475,0.013912,0.360505,0.130774,39.112195,39.0,1.759579,0.0
28,fitness at iteration,4,NaN,NaN,NaN,NaN,94.979273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
29,0.500000,0.575672,0.789294,5.0,0.000000,1567.873030,0.000000,0.01225,0.012011,0.300399,0.120051,41.046820,39.0,1.554744,0.0
30,0.653712,0.341427,0.679610,4.0,0.000000,1067.935291,0.000000,0.01350,0.010943,0.358146,0.144652,36.553568,39.0,1.475915,0.0
31,0.719886,0.3,1.095408,3.0,0.000000,719.974414,0.000000,0.01475,0.011256,0.671713,0.210468,27.138098,39.0,2.191521,0.0
32,0.510525,0.740138,0.916630,3.0,0.000000,1401.136363,0.000000,0.01475,0.013889,0.277658,0.139537,37.453808,39.0,0.989852,0.0
33,0.702697,0.54701,0.924501,3.0,0.000000,784.002429,0.000000,0.01475,0.011673,0.327702,0.196249,28.845896,39.0,0.669828,0.0
34,fitness at iteration,5,NaN,NaN,NaN,NaN,94.979273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
35,Best Solution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,0.747058,0.54701,0.924501,3.0,94.979273,699.566324,0.000000,0.01475,0.011320,0.320449,0.208468,27.364153,39.0,0.537162,False


# best result file

create file

In [30]:
dir_name = dir_to_save

# -- functions to save a file and create a dir --
def create_file(text):
    filename = dir_name+'/' + text +'.csv'
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        header = ["D = propeller diameter [m]",
                  "AEdAO = expanded area ratio",
                  "PdD = pitch ratio",
                  "Z = propeller's number of blades",
                  "P_B = power brake",
                  "n = Propeller angular speed [rpm]",
                  "fitness",
                  "i = thread pool id"]
        writer.writerow(header)
    return filename

def append_to_file(filename, row):
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)
        
def append_to_file_order(filename, D='', AEdAO='', PdD='', Z='', P_B='', n='', fitness='', i=''):
    row = [D, AEdAO, PdD, Z, P_B, n, fitness, i]
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

def save_best_result(D, Z, AEdAO, PdD, fitness, seed, solver_name, P_B='', n='', history=''):
    # create the csv file with the headers
    fitness = -fitness
    Z = int(Z)
    filename = create_file('best_results_' + str(seed) + '_' + solver_name)
    append_to_file_order(filename, D=D, AEdAO=AEdAO, PdD=PdD, Z=Z, P_B=P_B, n=P_B, fitness=fitness)
    append_to_file(filename, ['history'])
    append_to_file(filename, history)
    print('saved', filename)

In [31]:
D       = float(result_row.iloc[0]['D'])
Z       = int(result_row.iloc[0]['Z'])
AEdAO   = float(result_row.iloc[0]['AEdAO'])
PdD     = float(result_row.iloc[0]['PdD'])
fitness = float(result_row.iloc[0]['fitness'])
P_B = float(result_row.iloc[0]['P_B'])
n = float(result_row.iloc[0]['n'])

# history
filename = 'df_' + str(seed) + '.csv'
history = []
with open(filename, 'r', newline='') as file:
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        if 'best' in row[0]:
            # iteration = row[0].split(' ')[-1]
            fit = -float(row[1])
            history.append(str(fit))

save_best_result(D, Z, AEdAO, PdD, fitness, seed, 'original', P_B=P_B, n=n, history=history)

saved original/best_results_1_original.csv
